In [2]:
# import os

# os.chdir('/Users/paulynnyu/twitter_churn/tweepy-bots')


In [3]:
# !pip install tweepy
# !/usr/bin/env python tweepy-bots/bots/followfollowers.py

In [4]:
import tweepy
import json
from collections import defaultdict
import pandas as pd
import numpy as np
# tweepy-bots/bots/config.py

In [11]:
# config details
auth = tweepy.OAuthHandler("a4p9a6TTttFfvipx8P4YTSWKb", "QxGHVVzGhim9m50FQRvhSJNXlGkeXz5ZM5X5RpMPerjEPOGotH")
auth.set_access_token("2263992234-CnKivGAOr5oKszva3AelT4Aw0tUUzpAc7V8IuAb",
    "owyFowMakwBdaGbfZ86aQN0kRiZiH1DPyRIPdeJ1trIO2")

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True, parser=tweepy.parsers.JSONParser())



In [6]:
def get_ids(screen_name):
    '''
    :argument: screen_name of user
    :returns: a list_id of the given user's followers
    '''
    # get first list
    first_list = api.followers_ids(screen_name=screen_name)
    id_list = first_list['ids']
    cursor = first_list['next_cursor']

    while cursor != 0 :
        print(cursor)
        print(len(id_list))
        user_ids = api.followers_ids(screen_name=screen_name, cursor=cursor)
        id_list.extend(user_ids[0]['ids'])
        cursor = user_ids[0]['next_cursor']
    print(cursor)
#     print(user_ids)
    return id_list

In [7]:
id_list = get_ids("SFUnified")

1526113232950733589
5000
1387789003534045242
10000
0


In [21]:
len(id_list)

5002

In [17]:
len(id_list)

5002

In [5]:
# try:
#     api.verify_credentials()
#     print("Authentication OK")
# except:
#     print("Error during authentication")

Authentication OK


In [277]:
# Returns an array containing the IDs of users following the specified user.
user_ids = api.followers_ids(screen_name='OddStats')


In [284]:
len(user_ids['ids'])

5000

In [288]:
user_id2 = api.followers_ids(screen_name='OddStats',cursor=-2)

In [305]:
len(user_id2[0]['ids'])


5000

In [ ]:
user_ids = api.followers_ids(screen_name='OddStats')


In [ ]:
with open('Data/vegas_dataset.pickle','rb') as read_file:
    grouped_df = grouped_df.append(pickle.load(read_file))

In [317]:
# define screen_name
screen_name = 'paulynnyu'

# get first list
first_list = api.followers_ids(screen_name=screen_name)
id_list = first_list['ids']
cursor = first_list['next_cursor']

while cursor != 0:
    user_ids = api.followers_ids(screen_name=screen_name, cursor=cursor)
    id_list.append(user_ids[0]['ids'])
    cursor = user_ids[0]['next_cursor']

In [8]:
timeline = api.user_timeline(user_id=1177773206641340416, count=200)

In [ ]:
def get_tweets(user_id, timeline) :
    '''
    :param user_id: the list_id as recognized by twitter
    :param timeline: the dictionary that is pulled from user_timeline twitter API
    :return: all the posts in dictionary format of a specified list_id
    '''
    # attribute lists
    post_attrs = ['created_at', 'text', 'source', 'in_reply_to_screen_name', 'retweet_count', 'favorite_count',
                  'favorited', 'retweeted', 'is_quote_status', 'retweeted_status']
    entities_attrs = ['hashtags', 'symbols', 'user_mentions']

    # creating empty dictionary, and specifying user id
    post_dict = defaultdict(list)

    for tweets in timeline :
        post_dict['user_id'] = user_id
        for post in list(post_attrs) :
            if post == 'text' :
                try :
                    t = tweets[post]
                    t = t.replace('\n','')
                    t = t.replace('\r','')
                    post_dict[post].append(t)
                except :
                    post_dict[post].append(np.nan)
            else :
                try :
                    t = tweets[post]
                    post_dict[post].append(t)
                except :
                    post_dict[post].append(np.nan)

         # looping through other post attributes
        for entity in entities_attrs :
            try :
                attr_name = 'len_' + entity
                post_dict[entity].append(tweets['entities'][entity])
                post_dict[attr_name].append(len(tweets['entities'][entity]))
            except :
                post_dict[entity].append(np.nan)
    return post_dict

In [7]:
def get_details(user_list):
    # list of user attributes and post attributes we want to collect
    attributes = ['name', 'screen_name', 'location', 'description', 'url', 'protected', 'followers_count', 
                 'following', 'friends_count', 'listed_count', 'geo_enabled', 'created_at', 'favourites_count',
                 'statuses_count', 'contributors_enabled', 'is_translation_enabled', 'has_extended_profile',
                 'notifications', 'profile_image_url', 'profile_use_background_image']
    post_attrs = ['created_at', 'text', 'source', 'in_reply_to_screen_name', 'retweet_count', 'favorite_count',
               'favorited', 'retweeted', 'is_quote_status', 'retweeted_status']
    entities_attrs = ['hashtags', 'symbols', 'user_mentions']
    users_dict = []
    posts_dict = []
    
    # start looping through our list
    for users in user_list:
        print(users)
        user_dict = defaultdict(list)
        post_dict = defaultdict(list)
        user_dict['user_id'] = users

        try:
            timeline = api.user_timeline(user_id=users, count=200) # call api

            user_info = timeline[0]['user'] # creating a user_info and post dicts

            # looping through each user attribute
            for attribute in attributes:
                try:
                    user_dict[attribute].append(user_info[attribute])

                except:
                    user_dict[attribute].append(np.nan)

            # looping through each post, and each post attribute 
            for tweets in timeline:
                post['user_id'] = user['user_id']

                # looping through entities attributes
                for attr in entities_attrs:
                    try:
                        post_dict[attr].append(tweets['entities'][attr])
                        attr_name = 'len_' + attr
                        post_dict[attr_name].append(len(tweets['entities'][attr]))
                    except:
                        post_dict[attr].append(np.nan)

                # looping through other post attributes
                for attr in post_attrs:
                    try:
                        post_dict[attr].append(tweets[attr])
                    except:
                        post_dict[attr].append(np.nan)
                        
            posts_dict.append(post_dict)
            
        except:
            user_dict['not_authorized'].append(1)
    
        
            print(pd.DataFrame(user_dict))
            

    return users_dict, posts_dict


In [86]:
attributes = ['name', 'screen_name', 'location', 'description', 'url', 'protected', 'followers_count', 
             'following', 'friends_count', 'listed_count', 'geo_enabled', 'created_at', 'favourites_count',
             'statuses_count', 'contributors_enabled', 'is_translation_enabled', 'has_extended_profile',
             'notifications', 'profile_image_url', 'profile_use_background_image']

users_dict = []
posts_dict = []

# start looping through our list
for users in id_list:
#     print(users)
    user_dict = defaultdict(list)
    user_dict['user_id'] = users

    try:
        timeline = api.user_timeline(user_id=users, count=200) # call api

        user_info = timeline[0]['user'] # creating a user_info and post dicts

        # looping through each user attribute
        for attribute in attributes:
            try:
                user_dict[attribute].append(user_info[attribute])

            except:
                user_dict[attribute].append(np.nan)
                
        users_dict.append(user_dict)

#         looping through each post, and each post attribute 
        post_dict = defaultdict(list)
    
        for tweets in timeline:
            post_dict['user_id'] = user['user_id']


    # looping through entities attributes
            for entity in entities_attrs:
                print(entity)
                try:
                    post_dict[attr].append(tweets['entities'][attr])
                    attr_name = 'len_' + attr
                    post_dict[attr_name].append(len(tweets['entities'][attr]))
                except:
                    post_dict[attr].append(np.nan)

#             # looping through other post attributes
            for post in list(post_attrs):
                try:
                    post_dict[attr].append(tweets[attr])
                except:
                    post_dict[attr].append(np.nan)

#             print(post_dict)

    except:
        user_dict['not_authorized'].append(1)
    
    

#     print(pd.DataFrame(user_dict))


In [87]:
post_dict

defaultdict(list, {})

In [132]:
user_id = 11213442
timeline = api.user_timeline(user_id=user_id, count=200)

In [133]:
len(timeline)

200

In [178]:
id_list = ['11213442','243471552','539823113','794771306']

In [319]:
def get_tweets(user_id, timeline): 
    # attribute lists
    post_attrs = ['created_at', 'text', 'source', 'in_reply_to_screen_name', 'retweet_count', 'favorite_count',
           'favorited', 'retweeted', 'is_quote_status', 'retweeted_status']
    entities_attrs = ['hashtags', 'symbols', 'user_mentions']

    # creating empty dictionary, and specifying user id
    post_dict = defaultdict(list)   
    
    for tweets in timeline:
        post_dict['user_id'] = user_id
        for post in list(post_attrs):
            try:
                post_dict[post].append(tweets[post])
            except:
                post_dict[post].append(np.nan)   
            
        # looping through other post attributes
        for entity in entities_attrs:
            try:
                attr_name = 'len_' + entity
                post_dict[entity].append(tweets['entities'][entity])
                post_dict[attr_name].append(len(tweets['entities'][entity]))
            except:
                post_dict[entity].append(np.nan)
    return post_dict
        

In [320]:
def get_users(user_id, timeline):
    # list of attributes
    attributes = ['name', 'screen_name', 'location', 'description', 'url', 'protected', 'followers_count', 
             'following', 'friends_count', 'listed_count', 'geo_enabled', 'created_at', 'favourites_count',
             'statuses_count', 'contributors_enabled', 'is_translation_enabled', 'has_extended_profile',
             'notifications', 'profile_image_url', 'profile_use_background_image']
    
    user_dict = defaultdict(list) # creating empty dictionary, and specifying user id   
    user_dict['user_id'] = user_id
    user_info = timeline[0]['user'] # creating a user_info and post dicts

    # looping through each user attribute
    for attribute in attributes:
        try:
            user_dict[attribute].append(user_info[attribute])

        except:
            user_dict[attribute].append(np.nan)

    return user_dict

In [325]:
posts_dict = pd.DataFrame()
users_dict = pd.DataFrame()

for users in id_list:
    try:
        timeline = api.user_timeline(user_id=users, count=200)
        tweets = get_tweets(users, timeline)
        users = get_users(users, timeline)
                   
    except:
        users = defaultdict(list)
        tweets = defaultdict(list)
        users['user_id'].append(users)
        users['authorized'].append(1)

    # print full list
    users_dict = users_dict.append(pd.DataFrame(users))
    posts_dict = posts_dict.append(pd.DataFrame(tweets))


66049693
804974102024814592
defaultdict(<class 'list'>, {})
798649268500242432
11213442
414133340
3285483552
2795213426
756139014
59913032
317566549
3246459237
244632386
defaultdict(<class 'list'>, {})
2359684681
1377373945
2347766454
2920037648
defaultdict(<class 'list'>, {})
2881941724
564898014
49602043
243471552
539823113
defaultdict(<class 'list'>, {})
794771306
54689188
146674289
2458785205
394794278
defaultdict(<class 'list'>, {})
128112849
224331350
108025539
defaultdict(<class 'list'>, {})
145345389
39650205


In [324]:
users_dict

,authorized,contributors_enabled,created_at,description,favourites_count,followers_count,following,friends_count,geo_enabled,has_extended_profile,...,location,name,notifications,profile_image_url,profile_use_background_image,protected,screen_name,statuses_count,url,user_id
0,NaN,False,Sun Aug 16 04:14:56 +0000 2009,"Middle Market Coverage Banker, Citibank N.A., ...",159.0,54.0,False,640.0,False,False,...,San Francisco,Shyam Desai,False,http://pbs.twimg.com/profile_images/1888268742...,False,False,desai2702,182.0,None,66049693
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'user_id': [{'user_id': [defaultdict(<class '...
0,NaN,False,Tue Nov 15 22:09:58 +0000 2016,Wharton Alumni Angels 😇,49.0,185.0,False,302.0,False,False,...,"Palo Alto, CA",WhartonAlumniAngels,False,http://pbs.twimg.com/profile_images/1060798998...,False,False,wharangels,177.0,https://t.co/p23Z8MiKBY,798649268500242432
0,NaN,False,Sun Dec 16 04:08:15 +0000 2007,,4.0,133.0,False,218.0,False,False,...,"ÜT: 34.022772,-118.511242",Trevor,False,http://pbs.twimg.com/profile_images/7450222029...,True,False,teknojazdj,263.0,None,11213442
0,NaN,False,Wed Nov 16 17:34:22 +0000 2011,,4.0,47.0,False,155.0,False,False,...,France,Hugo,False,http://pbs.twimg.com/profile_images/8339878244...,False,False,Skyangelo_,105.0,None,414133340
0,NaN,False,Mon Jul 20 15:22:49 +0000 2015,"The PaymentsEd Forum delivers high-level, educ...",323.0,1219.0,False,3188.0,False,False,...,,Payments Ed,False,http://pbs.twimg.com/profile_images/7804473974...,False,False,PaymentsEd,1008.0,https://t.co/C4RXndN2JZ,3285483552
0,NaN,False,Sun Sep 07 03:11:22 +0000 2014,"@Cornell Alum; @SAP, Business and IT Strategy,...",10539.0,6205.0,False,7106.0,True,False,...,"Pittsburgh, PA","Indraneel Chatterjee, MBA",False,http://pbs.twimg.com/profile_images/7406221195...,False,False,Neel__C,76072.0,https://t.co/jUn1BZbRA4,2795213426
0,NaN,False,Tue Aug 14 00:46:08 +0000 2012,,0.0,5.0,False,13.0,False,False,...,,Jessie,False,http://abs.twimg.com/sticky/default_profile_im...,True,False,jessie_jlee,2.0,None,756139014
0,NaN,False,Fri Jul 24 23:01:17 +0000 2009,#Business news and insights for and about alum...,3098.0,17519.0,True,1095.0,True,False,...,"Philadelphia, PA",Wharton Magazine,False,http://pbs.twimg.com/profile_images/1119273118...,False,False,whartonmagazine,12471.0,http://t.co/BFr2CpSS11,59913032
0,NaN,False,Wed Jun 15 03:46:20 +0000 2011,#Finance Professional @Vitalyst; Organizer @Li...,38.0,26.0,False,119.0,False,False,...,"Philadelphia, PA",Jeffrey Payne,False,http://pbs.twimg.com/profile_images/6091872295...,False,False,Jeff_C_Payne,175.0,None,317566549


In [204]:
pd.DataFrame(posts_dict)

,created_at,text,source,in_reply_to_screen_name,retweet_count,favorite_count,favorited,retweeted,is_quote_status,retweeted_status


In [162]:
pd.DataFrame(post_dict)s

SyntaxError: invalid syntax (<ipython-input-162-441cb069fe3c>, line 1)

In [83]:
for value in post_dict:
    print(value)

user_mentions
len_user_mentions


In [84]:


try:
    user['name'].append(timeline[0]['user']['name'])
    user['screen_name'].append(timeline[0]['user']['screen_name'])
    user['location'].append(timeline[0]['user']['location'])
    user['description'].append(timeline[0]['user']['description'])
    user['url'].append(timeline[0]['user']['url'])
    user['protected'].append(timeline[0]['user']['protected'])
    user['followers_cnt'].append(timeline[0]['user']['followers_count'])
    user['following_cnt'].append(timeline[0]['user']['following'])
    user['friends_cnt'].append(timeline[0]['user']['friends_count'])
    user['listed_cnt'].append(timeline[0]['user']['listed_count'])
    user['geo_enabled'].append(timeline[0]['user']['geo_enabled'])
    user['created_at'].append(timeline[0]['user']['created_at'])
    user['favorites_cnt'].append(timeline[0]['user']['favourites_count'])
    user['statuses_cnt'].append(timeline[0]['user']['statuses_count'])
    user['contributors_enabled'].append(timeline[0]['user']['contributors_enabled'])
    user['translation_enabled'].append(timeline[0]['user']['is_translation_enabled'])
    user['extended_profile'].append(timeline[0]['user']['has_extended_profile'])
    user['notifications'].append(timeline[0]['user']['notifications'])
    user['profile_image'].append(timeline[0]['user']['profile_image_url'])
    user['profile_background_YN'].append(timeline[0]['user']['profile_use_background_image'])



    post = defaultdict(list)
    for tweets in timeline:
    # getting first details

    
    
#     print(tweet)
        post['user_id'] = user['user_id']
        post['date'].append(tweets['created_at'])
        post['text'].append(tweets['text'])
        post['hashtags'].append(tweets['entities']['hashtags'])
        post['len_hashtags'].append(len(tweets['entities']['hashtags']))
        post['symbols'].append(tweets['entities']['symbols'])
        post['len_symbols'].append(len(tweets['entities']['symbols']))
        post['len_user_mentions'].append(len(tweets['entities']['user_mentions']))
        post['source'].append(tweets['source'])
        post['reply_to'].append(tweets['in_reply_to_screen_name'])
        post['retweet'].append(tweets['retweet_count'])
        post['favorite_cnt'].append(tweets['favorite_count'])
        post['favorited'].append(tweets['favorited'])
        post['retweeted'].append(tweets['retweeted'])
        post['is_quote_status'].append(tweets['is_quote_status'])
        if tweets.get('retweeted_status'," "):
            post['retweet_status_YN'].append(len(tweets.get('retweeted_status'," ")) >2)
    #         post['len_retweet_content'].append(tweets.get('retweeted_status'," "))

    post = pd.DataFrame(post)
    user = pd.DataFrame(user)

except:
    user['empty'].append(1)


# print(pd.DataFrame(post))

# print(pd.DataFrame(user))


NameError: name 'user' is not defined